# 🧩 CIFAR-10 Autoencoder Project

This notebook demonstrates the use of an Autoencoder for dimensionality reduction and image reconstruction on the CIFAR-10 dataset. The model is trained to compress and then reconstruct images, learning meaningful patterns in an unsupervised fashion. Let's dive in! 🚀

In [ ]:

# Import required libraries 📚
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import torch.utils.data as data
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt  # For plotting 🖼️

# Setting random seed for reproducibility 🔄
SEED = 87
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)


In [ ]:

def print_model(encoder, decoder):
    """Prints the architecture of the encoder and decoder models."""
    print("============== Encoder ==============")
    print(encoder)
    print("============== Decoder ==============")
    print(decoder)
    print("")


In [ ]:

def create_model():
    """Creates and initializes the Autoencoder model."""
    autoencoder = Autoencoder()
    print_model(autoencoder.encoder, autoencoder.decoder)
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU to speed up training. 🚀")
    return autoencoder


In [ ]:
def create_model1(encoder):
    autoencoder = AutoEncoder1(encoder)
    print_model(autoencoder.encoder, autoencoder.decoder)
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU in order to speed up training.")
    return autoencoder

In [ ]:

def get_torch_vars(x):
    """Moves the tensor to GPU if available."""
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)


In [ ]:
def imshow(img):
    npimg = img.cpu().detach().numpy()
    plt.axis('off')
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Input size: [batch, 3, 32, 32]
        # Output size: [batch, 3, 32, 32]
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),            # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),           # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.Conv2d(24, 48, 4, stride=2, padding=1),           # [batch, 48, 4, 4]
            nn.ReLU(),
# 			nn.Conv2d(48, 96, 4, stride=2, padding=1),           # [batch, 96, 2, 2]
#             nn.ReLU(),
        )
        self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(96, 48, 4, stride=2, padding=1),  # [batch, 48, 4, 4]
#             nn.ReLU(),
			nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),   # [batch, 3, 32, 32]
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
# Create model
autoencoder = create_model()

# Load data
transform = transforms.Compose(
    [transforms.ToTensor()
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Run this if you have pretrained weights

In [ ]:
print("Loading checkpoint...")
autoencoder.load_state_dict(torch.load("/kaggle/input/cifar10-encoder/weights/autoencoder.pkl"))
dataiter = iter(testloader)
images, labels = dataiter.next()
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(16)))
imshow(torchvision.utils.make_grid(images))

images = Variable(images.cuda())

decoded_imgs = autoencoder(images)[1]
imshow(torchvision.utils.make_grid(decoded_imgs.data))

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters())
epochs = 10

In [ ]:
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, _) in enumerate(trainloader, 0):
        inputs = get_torch_vars(inputs)

        # ============ Forward ============
        encoded, outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(autoencoder.state_dict(), "./weights/autoencoder.pkl")

In [ ]:
class CustomDataset(data.Dataset):
    def __init__(self, size=1000, transform=None):
        indexes = random.sample(range(0, 50000), size)
        group1 = indexes[:size//2]
        group2 = indexes[size//2:]
        self.train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
        self.indexes = []
        for i in group1:
            for j in group2:
                self.indexes.append((i, j))
        
    def __len__(self):
        return len(self.indexes)
    
    def __getitem__(self, idx):
        index1, index2 = self.indexes[idx]
        img1 = self.train_set[index1][0]
        img2 = self.train_set[index2][0]
        result = img1 + img2
        result = result / 2
        return result, (img1, img2)
        

In [ ]:
custom_set = CustomDataset(1000)
custom_loader = torch.utils.data.DataLoader(custom_set, batch_size=16,
                                          shuffle=True, num_workers=2)

In [ ]:
class AutoEncoder1(nn.Module):
    def __init__(self, encoder):
        super(AutoEncoder1, self).__init__()
        self.encoder = encoder
        self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(96, 48, 4, stride=2, padding=1),  # [batch, 48, 4, 4]
#             nn.ReLU(),
			nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),   # [batch, 3, 32, 32]
            nn.Sigmoid(),
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
# Create model
autoencoder1 = create_model1(autoencoder.encoder)

In [ ]:
for i, param in enumerate(autoencoder1.parameters()):
    print(i, param.size(), param.requires_grad)

In [ ]:
for i, param in enumerate(autoencoder1.parameters()):
    if i < 6:
        param.requires_grad = False

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, autoencoder1.parameters()))
epochs = 20

In [ ]:
print("Loading checkpoint...")
autoencoder1.load_state_dict(torch.load("/kaggle/input/cifar10-encoder/weights/autoencoder1.pkl"))

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(custom_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    imshow(inputs[0])
    imshow(outputs1[0])
    imshow(outputs2[0])
    break

In [ ]:
autoencoder1

In [ ]:
epoch_counter = 0
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, (outputs1, outputs2)) in enumerate(custom_loader, 0):
        inputs = get_torch_vars(inputs)
        outputs1 = get_torch_vars(outputs1)
        outputs2 = get_torch_vars(outputs2)
        # ============ Forward ============
        encoded, outputs = autoencoder1(inputs)
        loss = criterion(outputs, outputs1)
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    if epoch_counter == 4:
        epoch_counter = -1
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(autoencoder1.state_dict(), "./weights/autoencoder1.pkl")
    epoch_counter += 1
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(autoencoder1.state_dict(), "./weights/autoencoder1.pkl")

In [ ]:
autoencoder1

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(custom_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("first image: ")
    imshow(outputs1[0])
    print("second image: ")
    imshow(outputs2[0])
    print("predicted image: ")
    imshow(autoencoder1(inputs[0])[1])
    break

In [ ]:
test_set = CustomDataset(1000)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=16,
                                          shuffle=True, num_workers=2)

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("first image: ")
    imshow(outputs1[0])
    print("second image: ")
    imshow(outputs2[0])
    print("predicted image: ")
    imshow(autoencoder1(inputs[0])[1])
    break

In [ ]:
autoencoder2 = create_model1(autoencoder.encoder)

In [ ]:
print("Loading checkpoint...")
autoencoder2.load_state_dict(torch.load("/kaggle/input/cifar10-encoder/weights/autoencoder2.pkl"))

In [ ]:
for i, param in enumerate(autoencoder2.parameters()):
    if i < 6:
        param.requires_grad = False

In [ ]:
for i, param in enumerate(autoencoder2.parameters()):
    print(i, param.size(), param.requires_grad)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, autoencoder2.parameters()))
epochs = 50

In [ ]:
epoch_counter = 0
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, (outputs1, outputs2)) in enumerate(custom_loader, 0):
        inputs = get_torch_vars(inputs)
        outputs1 = get_torch_vars(outputs1)
        outputs2 = get_torch_vars(outputs2)
        # ============ Forward ============
        encoded, outputs = autoencoder2(inputs)
        loss = criterion(outputs, outputs2)
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    epoch_counter += 1
    if epoch_counter == 4:
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(autoencoder2.state_dict(), "./weights/autoencoder2.pkl")
        epoch_counter = -1
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(autoencoder2.state_dict(), "./weights/autoencoder2.pkl")

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("first image: ")
    imshow(outputs1[0])
    print("second image: ")
    imshow(outputs2[0])
    print("predicted image: ")
    imshow(autoencoder2(inputs[0])[1])
    break

In [ ]:
class MainAutoEncoder(nn.Module):
    def __init__(self, encoder, decoder1, decoder2):
        super(MainAutoEncoder, self).__init__()
        self.encoder = encoder
        self.decoder1 = decoder1
        self.decoder2 = decoder2
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded1 = self.decoder1(encoded)
        decoded2 = self.decoder2(encoded)
        return encoded, decoded1, decoded2

In [ ]:
def create_main_model(encoder, decoder1, decoder2):
    autoencoder = MainAutoEncoder(encoder, decoder1, decoder2)
#     print_model(autoencoder.encoder, autoencoder.decoder1, autoencoder.decoder2)
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU in order to speed up training.")
    return autoencoder

In [ ]:
main_model = create_main_model(autoencoder.encoder, autoencoder1.decoder, autoencoder2.decoder)
main_model

In [ ]:
print("Loading checkpoint...")
main_model.load_state_dict(torch.load("/kaggle/input/cifar10-encoder/weights/main_model.pkl"))

In [ ]:
for index, param in enumerate(main_model.parameters()):
    print(index, param.size(), param.requires_grad)

In [ ]:
for index, param in enumerate(main_model.parameters()):
    param.requires_grad = True

In [ ]:
for index, param in enumerate(main_model.parameters()):
    print(index, param.size(), param.requires_grad)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, main_model.parameters()))
epochs = 50

In [ ]:
epoch_counter = 0
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, (outputs1, outputs2)) in enumerate(custom_loader, 0):
        inputs = get_torch_vars(inputs)
        outputs1 = get_torch_vars(outputs1)
        outputs2 = get_torch_vars(outputs2)
        # ============ Forward ============
        encoded, output1, output2 = main_model(inputs)
        loss = (criterion(output1, outputs1) + criterion(output2, outputs2)) / 2
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    epoch_counter += 1
    if epoch_counter == 4:
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(main_model.state_dict(), "./weights/main_model.pkl")
        epoch_counter = -1
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(main_model.state_dict(), "./weights/main_model.pkl")

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("first image: ")
    imshow(outputs1[0])
    print("second image: ")
    imshow(outputs2[0])
    print("predicted image 1: ")
    imshow(main_model(inputs[0])[1])
    print("predicted image 2: ")
    imshow(main_model(inputs[0])[2])
    break

### Idea 2:

Customise encoder and decoder for denoising mean image and convert it to first image. Then we use $x_2 = 2 \times \frac{x_1 + x_2}{2} - x_1$ to discover second image

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Input size: [batch, 3, 32, 32]
        # Output size: [batch, 3, 32, 32]
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),            # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),           # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.Conv2d(24, 48, 4, stride=2, padding=1),           # [batch, 48, 4, 4]
            nn.ReLU(),
# 			nn.Conv2d(48, 96, 4, stride=2, padding=1),           # [batch, 96, 2, 2]
#             nn.ReLU(),
        )
        self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(96, 48, 4, stride=2, padding=1),  # [batch, 48, 4, 4]
#             nn.ReLU(),
			nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),   # [batch, 3, 32, 32]
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
model2 = create_model()
model2

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model2.parameters())
epochs = 50

In [ ]:
epoch_counter = 0
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, (outputs1, outputs2)) in enumerate(custom_loader, 0):
        inputs = get_torch_vars(inputs)
        outputs1 = get_torch_vars(outputs1)
        # ============ Forward ============
        encoded, outputs = model2(inputs)
        loss = criterion(outputs, outputs1)
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    epoch_counter += 1
    if epoch_counter == 4:
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(model2.state_dict(), "./weights/model2.pkl")
        epoch_counter = -1
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(model2.state_dict(), "./weights/model2.pkl")

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("first image: ")
    imshow(outputs1[0])
    print("second image: ")
    imshow(outputs2[0])
    print("predicted image 1: ")
    imshow(main_model(inputs[0])[1])
    print("predicted image 2: ")
    imshow(main_model(inputs[0])[2])
    break

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("first image: ")
    imshow(outputs1[0])
    print("second image: ")
    imshow(outputs2[0])
    print("predicted image 1: ")
    imshow(model2(inputs[0])[1])
    print("predicted image 2: ")
    imshow(2 * inputs[0] - model2(inputs[0])[1])
    break

# Increase Latent Space Dimension

In [ ]:
class BiggerAutoencoder(nn.Module):
    def __init__(self):
        super(BiggerAutoencoder, self).__init__()
        # Input size: [batch, 3, 32, 32]
        # Output size: [batch, 3, 32, 32]
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),            # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),           # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.Conv2d(24, 48, 4, stride=2, padding=1),           # [batch, 48, 4, 4]
            nn.ReLU(),
			nn.Conv2d(48, 96, 4, stride=2, padding=1),           # [batch, 96, 2, 2]
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(96, 48, 4, stride=2, padding=1),  # [batch, 48, 4, 4]
            nn.ReLU(),
			nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),   # [batch, 3, 32, 32]
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
def create_bigger_model():
    autoencoder = BiggerAutoencoder()
    print_model(autoencoder.encoder, autoencoder.decoder)
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU in order to speed up training.")
    return autoencoder

In [ ]:
bigger_encoder = create_bigger_model()

# Load data
transform = transforms.Compose(
    [transforms.ToTensor()
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

custom_set = CustomDataset(1000)
custom_loader = torch.utils.data.DataLoader(custom_set, batch_size=16,
                                          shuffle=True, num_workers=2)

test_set = CustomDataset(1000)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=16,
                                          shuffle=True, num_workers=2)


In [ ]:
print("Loading checkpoint...")
bigger_encoder.load_state_dict(torch.load("/kaggle/input/cifar10-encoder/weights/bigger_encoder.pkl"))

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(bigger_encoder.parameters())
epochs = 5

In [ ]:
epoch_counter = 0
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, _) in enumerate(custom_loader, 0):
        inputs = get_torch_vars(inputs)
        # ============ Forward ============
        encoded, outputs = bigger_encoder(inputs)
        loss = criterion(outputs, inputs)
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    epoch_counter += 1
    if epoch_counter == 4:
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(bigger_encoder.state_dict(), "./weights/bigger_encoder.pkl")
        epoch_counter = -1
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(bigger_encoder.state_dict(), "./weights/bigger_encoder.pkl")

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("predicted image 1: ")
    imshow(bigger_encoder(inputs[0])[1])
    break

In [ ]:
class BiggerAutoEncoder1(nn.Module):
    def __init__(self, encoder):
        super(BiggerAutoEncoder1, self).__init__()
        self.encoder = encoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(96, 48, 4, stride=2, padding=1),  # [batch, 48, 4, 4]
            nn.ReLU(),
			nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),   # [batch, 3, 32, 32]
            nn.Sigmoid(),
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
def create_bigger_model1(encoder):
    autoencoder = BiggerAutoEncoder1(encoder)
    print_model(autoencoder.encoder, autoencoder.decoder)
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU in order to speed up training.")
    return autoencoder

In [ ]:
big_autoencoder1 = create_bigger_model1(bigger_encoder.encoder)

In [ ]:
print("Loading checkpoint...")
big_autoencoder1.load_state_dict(torch.load("/kaggle/input/cifar10-encoder/weights/big_autoencoder1.pkl"))

In [ ]:
for index, param in enumerate(big_autoencoder1.parameters()):
    print(index, param.size(), param.requires_grad)

In [ ]:
for index, param in enumerate(big_autoencoder1.parameters()):
    if index < 8:
        param.requires_grad = False

In [ ]:
for index, param in enumerate(big_autoencoder1.parameters()):
    print(index, param.size(), param.requires_grad)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, big_autoencoder1.parameters()))
epochs = 20

In [ ]:
epoch_counter = 0
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, (outputs1, outputs2)) in enumerate(custom_loader, 0):
        inputs = get_torch_vars(inputs)
        outputs1 = get_torch_vars(outputs1)
        outputs2 = get_torch_vars(outputs2)
        # ============ Forward ============
        encoded, outputs = big_autoencoder1(inputs)
        loss = criterion(outputs, outputs1)
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    if epoch_counter == 4:
        epoch_counter = -1
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(big_autoencoder1.state_dict(), "./weights/big_autoencoder1.pkl")
    epoch_counter += 1
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(big_autoencoder1.state_dict(), "./weights/big_autoencoder1.pkl")

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("first image: ")
    imshow(outputs1[0])
    print("predicted image 1: ")
    imshow(big_autoencoder1(inputs[0])[1])
    break

In [ ]:
big_autoencoder2 = create_bigger_model1(bigger_encoder.encoder)

In [ ]:
for index, param in enumerate(big_autoencoder2.parameters()):
    print(index, param.size(), param.requires_grad)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, big_autoencoder2.parameters()))
epochs = 25

In [ ]:
epoch_counter = 0
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, (outputs1, outputs2)) in enumerate(custom_loader, 0):
        inputs = get_torch_vars(inputs)
        outputs1 = get_torch_vars(outputs1)
        outputs2 = get_torch_vars(outputs2)
        # ============ Forward ============
        encoded, outputs = big_autoencoder2(inputs)
        loss = criterion(outputs, outputs2)
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    if epoch_counter == 4:
        epoch_counter = -1
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(big_autoencoder2.state_dict(), "./weights/big_autoencoder2.pkl")
    epoch_counter += 1
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(big_autoencoder2.state_dict(), "./weights/big_autoencoder2.pkl")

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("second image: ")
    imshow(outputs2[0])
    print("predicted image 1: ")
    imshow(big_autoencoder2(inputs[0])[1])
    break

In [ ]:
class BigMainAutoEncoder(nn.Module):
    def __init__(self, encoder, decoder1, decoder2):
        super(BigMainAutoEncoder, self).__init__()
        self.encoder = encoder
        self.decoder1 = decoder1
        self.decoder2 = decoder2
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded1 = self.decoder1(encoded)
        decoded2 = self.decoder2(encoded)
        return encoded, decoded1, decoded2

In [ ]:
def create_big_main_model(encoder, decoder1, decoder2):
    autoencoder = BigMainAutoEncoder(encoder, decoder1, decoder2)
#     print_model(autoencoder.encoder, autoencoder.decoder1, autoencoder.decoder2)
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU in order to speed up training.")
    return autoencoder

In [ ]:
main_model = create_big_main_model(bigger_encoder.encoder, big_autoencoder1.decoder, big_autoencoder2.decoder)
main_model

In [ ]:
for index, param in enumerate(main_model.parameters()):
    print(index, param.size(), param.requires_grad)

In [ ]:
for index, param in enumerate(main_model.parameters()):
    param.requires_grad = True

In [ ]:
for index, param in enumerate(main_model.parameters()):
    print(index, param.size(), param.requires_grad)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, main_model.parameters()))
epochs = 25

In [ ]:
epoch_counter = 0
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, (outputs1, outputs2)) in enumerate(custom_loader, 0):
        inputs = get_torch_vars(inputs)
        outputs1 = get_torch_vars(outputs1)
        outputs2 = get_torch_vars(outputs2)
        # ============ Forward ============
        encoded, output1, output2 = main_model(inputs)
        loss = (criterion(output1, outputs1) + criterion(output2, outputs2)) / 2
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    epoch_counter += 1
    if epoch_counter == 4:
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(main_model.state_dict(), "./weights/big_main_model.pkl")
        epoch_counter = -1
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(main_model.state_dict(), "./weights/big_main_model.pkl")

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("first image: ")
    imshow(outputs1[0])
    print("second image: ")
    imshow(outputs2[0])
    print("predicted image 1: ")
    imshow(main_model(inputs[0])[1])
    print("predicted image 2: ")
    imshow(main_model(inputs[0])[2])
    loss = (criterion(main_model(inputs[0])[1], outputs1[0]) + criterion(main_model(inputs[0])[2], outputs2[0])) / 2
    print("loss: ", loss)
    break

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("first image: ")
    imshow(outputs1[0])
    print("second image: ")
    imshow(outputs2[0])
    print("predicted image 1: ")
    imshow(main_model(inputs[0])[1])
    print("predicted image 2: ")
    imshow(main_model(inputs[0])[2])
    loss = (criterion(main_model(inputs[0])[1], outputs1[0]) + criterion(main_model(inputs[0])[2], outputs2[0])) / 2
    print("loss: ", loss)
    break

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("first image: ")
    imshow(outputs1[0])
    print("second image: ")
    imshow(outputs2[0])
    print("predicted image 1: ")
    imshow(main_model(inputs[0])[1])
    print("predicted image 2: ")
    imshow(main_model(inputs[0])[2])
    loss = (criterion(main_model(inputs[0])[1], outputs1[0]) + criterion(main_model(inputs[0])[2], outputs2[0])) / 2
    print("loss: ", loss)
    break

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("first image: ")
    imshow(outputs1[0])
    print("second image: ")
    imshow(outputs2[0])
    print("predicted image 1: ")
    imshow(main_model(inputs[0])[1])
    print("predicted image 2: ")
    imshow(main_model(inputs[0])[2])
    loss = (criterion(main_model(inputs[0])[1], outputs1[0]) + criterion(main_model(inputs[0])[2], outputs2[0])) / 2
    print("loss: ", loss)
    break

# Main big model with specific Encoder and Decoder

In [ ]:
bigger_encoder = create_bigger_model()

# Load data
transform = transforms.Compose(
    [transforms.ToTensor()
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

custom_set = CustomDataset(1000)
custom_loader = torch.utils.data.DataLoader(custom_set, batch_size=16,
                                          shuffle=True, num_workers=2)

test_set = CustomDataset(1000)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=16,
                                          shuffle=True, num_workers=2)


In [ ]:
print("Loading checkpoint...")
bigger_encoder.load_state_dict(torch.load("/kaggle/input/cifar10-encoder/weights/bigger_encoder.pkl"))

In [ ]:
big_autoencoder1 = create_bigger_model1(bigger_encoder.encoder)

In [ ]:
print("Loading checkpoint...")
big_autoencoder1.load_state_dict(torch.load("/kaggle/input/cifar10-encoder/weights/big_autoencoder1.pkl"))

In [ ]:
big_autoencoder2 = create_bigger_model1(bigger_encoder.encoder)

In [ ]:
print("Loading checkpoint...")
big_autoencoder2.load_state_dict(torch.load("/kaggle/input/cifar10-encoder/weights/big_autoencoder2.pkl"))

In [ ]:
main_model = create_big_main_model(bigger_encoder.encoder, big_autoencoder1.decoder, big_autoencoder2.decoder)
main_model

In [ ]:
print("Loading checkpoint...")
main_model.load_state_dict(torch.load("/kaggle/input/cifar10-encoder/weights/big_main_model.pkl"))

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, main_model.parameters()))
epochs = 25

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("first image: ")
    imshow(outputs1[0])
    print("second image: ")
    imshow(outputs2[0])
    print("predicted image 1: ")
    imshow(main_model(inputs[0])[1])
    print("predicted image 2: ")
    imshow(main_model(inputs[0])[2])
    loss = (criterion(main_model(inputs[0])[1], outputs1[0]) + criterion(main_model(inputs[0])[2], outputs2[0])) / 2
    print("loss: ", loss)
    break

In [ ]:
post_denoiser1 = create_bigger_model()

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(post_denoiser1.parameters())
epochs = 25

In [ ]:
epoch_counter = 0
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, (outputs1, outputs2)) in enumerate(custom_loader, 0):
        inputs = get_torch_vars(inputs)
        outputs1 = get_torch_vars(outputs1)
        outputs2 = get_torch_vars(outputs2)
        # ============ Forward ============
        encoded, main_output1, main_output2 = main_model(inputs)
        main_output1 = get_torch_vars(main_output1)
        denoise_encoded, denoised_output = post_denoiser1(main_output1[0])
        loss = criterion(denoised_output, outputs1)
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    if epoch_counter == 4:
        epoch_counter = -1
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(post_denoiser1.state_dict(), "./weights/post_denoiser1.pkl")
    epoch_counter += 1
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(post_denoiser1.state_dict(), "./weights/post_denoiser1.pkl")

# Modified Model 2

In [ ]:
class ConvAutoEncoder(nn.Module):
    def __init__(self):
        super(ConvAutoEncoder, self).__init__()
        # Input size: [batch, 3, 32, 32]
        # Output size: [batch, 3, 32, 32]
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 3, padding=1),            # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.Conv2d(12, 24, 3, padding=1),           # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.Conv2d(24, 48, 3, padding=1),           # [batch, 48, 4, 4]
            nn.ReLU(),
			nn.Conv2d(48, 96, 3, padding=1),           # [batch, 96, 2, 2]
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(96, 48, 3, padding=1),  # [batch, 48, 4, 4]
            nn.ReLU(),
			nn.ConvTranspose2d(48, 24, 3, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 3, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 3, padding=1),   # [batch, 3, 32, 32]
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
def create_bigger_model():
    autoencoder = ConvAutoEncoder()
    print_model(autoencoder.encoder, autoencoder.decoder)
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU in order to speed up training.")
    return autoencoder

In [ ]:
bigger_encoder = create_bigger_model()

# Load data
transform = transforms.Compose(
    [transforms.ToTensor()
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
    ])

custom_set = CustomDataset(1000, transform)
custom_loader = torch.utils.data.DataLoader(custom_set, batch_size=16,
                                          shuffle=True, num_workers=2)

test_set = CustomDataset(1000, transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=16,
                                          shuffle=True, num_workers=2)


In [ ]:
bigger_encoder.load_state_dict(torch.load("/kaggle/input/cifar10-encoder/weights/bigger_encoder.pkl"))

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(bigger_encoder.parameters())
epochs = 5

In [ ]:
epoch_counter = 0
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, _) in enumerate(custom_loader, 0):
        inputs = get_torch_vars(inputs)
        # ============ Forward ============
        encoded, outputs = bigger_encoder(inputs)
        loss = criterion(outputs, inputs)
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    epoch_counter += 1
    if epoch_counter == 4:
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(bigger_encoder.state_dict(), "./weights/3bigger_encoder.pkl")
        epoch_counter = -1
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(bigger_encoder.state_dict(), "./weights/3bigger_encoder.pkl")

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    print(inputs[0].std())
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("predicted image 1: ")
    imshow(bigger_encoder(inputs[0])[1])
    loss = criterion(bigger_encoder(inputs[0])[1], inputs[0])
    print("Loss:", loss.data)
    break

In [ ]:
class ConvAutoEncoder1(nn.Module):
    def __init__(self, encoder):
        super(ConvAutoEncoder1, self).__init__()
        self.encoder = encoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(96, 48, 3, padding=1),  # [batch, 48, 4, 4]
            nn.ReLU(),
			nn.ConvTranspose2d(48, 24, 3, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 3, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 3, padding=1),   # [batch, 3, 32, 32]
            nn.Sigmoid(),
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
def create_conv_model1(encoder):
    autoencoder = ConvAutoEncoder1(encoder)
    print_model(autoencoder.encoder, autoencoder.decoder)
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU in order to speed up training.")
    return autoencoder

In [ ]:
conv_autoencoder1 = create_conv_model1(bigger_encoder.encoder)

In [ ]:
for index, param in enumerate(conv_autoencoder1.parameters()):
    print(index, param.size(), param.requires_grad)

In [ ]:
for index, param in enumerate(conv_autoencoder1.parameters()):
    if index < 8:
        param.requires_grad = False

In [ ]:
for index, param in enumerate(conv_autoencoder1.parameters()):
    print(index, param.size(), param.requires_grad)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, conv_autoencoder1.parameters()))
epochs = 10

In [ ]:
epoch_counter = 0
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, (outputs1, outputs2)) in enumerate(custom_loader, 0):
        inputs = get_torch_vars(inputs)
        outputs1 = get_torch_vars(outputs1)
        outputs2 = get_torch_vars(outputs2)
        # ============ Forward ============
        encoded, outputs = conv_autoencoder1(inputs)
        loss = criterion(outputs, outputs1)
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    if epoch_counter == 4:
        epoch_counter = -1
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(conv_autoencoder1.state_dict(), "./weights/conv_autoencoder1.pkl")
    epoch_counter += 1
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(conv_autoencoder1.state_dict(), "./weights/conv_autoencoder1.pkl")

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    print(inputs[0].std())
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("Image 1: ")
    imshow(outputs1[0])
    print("predicted image 1: ")
    imshow(conv_autoencoder1(inputs[0])[1])
    loss = criterion(conv_autoencoder1(inputs[0])[1], outputs1[0])
    print("Loss:", loss.data)
    break

In [ ]:
conv_autoencoder2 = create_conv_model1(bigger_encoder.encoder)

In [ ]:
for index, param in enumerate(conv_autoencoder2.parameters()):
    print(index, param.size(), param.requires_grad)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, conv_autoencoder2.parameters()))
epochs = 10

In [ ]:
epoch_counter = 0
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, (outputs1, outputs2)) in enumerate(custom_loader, 0):
        inputs = get_torch_vars(inputs)
        outputs1 = get_torch_vars(outputs1)
        outputs2 = get_torch_vars(outputs2)
        # ============ Forward ============
        encoded, outputs = conv_autoencoder2(inputs)
        loss = criterion(outputs, outputs2)
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    if epoch_counter == 4:
        epoch_counter = -1
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(conv_autoencoder2.state_dict(), "./weights/conv_autoencoder2.pkl")
    epoch_counter += 1
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(conv_autoencoder2.state_dict(), "./weights/conv_autoencoder2.pkl")

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    print(inputs[0].std())
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("Image 2: ")
    imshow(outputs2[0])
    print("predicted image 2: ")
    imshow(conv_autoencoder2(inputs[0])[1])
    loss = criterion(conv_autoencoder2(inputs[0])[1], outputs2[0])
    print("Loss:", loss.data)
    break

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("Image 1: ")
    imshow(outputs1[0])
    print("Predicted image 1: ")
    imshow(conv_autoencoder1(inputs[0])[1])
    print("Image 2:")
    imshow(outputs2[0])
    print("Predicted image 2:")
    imshow(conv_autoencoder2(inputs[0])[1])
    loss = (criterion(conv_autoencoder1(inputs[0])[1], outputs1[0]) + criterion(conv_autoencoder2(inputs[0])[1], outputs2[0]))/2
    print("Loss:", loss.data)
    break

In [ ]:
class ConvMainAutoEncoder(nn.Module):
    def __init__(self, encoder, decoder1, decoder2):
        super(ConvMainAutoEncoder, self).__init__()
        self.encoder = encoder
        self.decoder1 = decoder1
        self.decoder2 = decoder2
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded1 = self.decoder1(encoded)
        decoded2 = self.decoder2(encoded)
        return encoded, decoded1, decoded2

In [ ]:
def create_conv_main_model(encoder, decoder1, decoder2):
    autoencoder = ConvMainAutoEncoder(encoder, decoder1, decoder2)
#     print_model(autoencoder.encoder, autoencoder.decoder1, autoencoder.decoder2)
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU in order to speed up training.")
    return autoencoder

In [ ]:
main_model = create_conv_main_model(bigger_encoder.encoder, conv_autoencoder1.decoder, conv_autoencoder2.decoder)
main_model

In [ ]:
for index, param in enumerate(main_model.parameters()):
    param.requires_grad = True

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(main_model.parameters())
epochs = 25

In [ ]:
epoch_counter = 0
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, (outputs1, outputs2)) in enumerate(custom_loader, 0):
        inputs = get_torch_vars(inputs)
        outputs1 = get_torch_vars(outputs1)
        outputs2 = get_torch_vars(outputs2)
        # ============ Forward ============
        encoded, output1, output2 = main_model(inputs)
        loss = (criterion(output1, outputs1) + criterion(output2, outputs2)) / 2
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    epoch_counter += 1
    if epoch_counter == 4:
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(main_model.state_dict(), "./weights/conv_main_model.pkl")
        epoch_counter = -1
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(main_model.state_dict(), "./weights/conv_main_model.pkl")

In [ ]:
for i, (inputs, (outputs1, outputs2)) in enumerate(test_loader, 0):
    inputs = get_torch_vars(inputs)
    outputs1 = get_torch_vars(outputs1)
    outputs2 = get_torch_vars(outputs2)
    print("mean image: ")
    imshow(inputs[0])
    print("first image: ")
    imshow(outputs1[0])
    print("second image: ")
    imshow(outputs2[0])
    print("predicted image 1: ")
    imshow(main_model(inputs[0])[1])
    print("predicted image 2: ")
    imshow(main_model(inputs[0])[2])
    loss = (criterion(main_model(inputs[0])[1], outputs1[0]) + criterion(main_model(inputs[0])[2], outputs2[0])) / 2
    print("loss: ", loss)
    break

# Deep Decoder

In [ ]:
class ConvAutoEncoder(nn.Module):
    def __init__(self):
        super(ConvAutoEncoder, self).__init__()
        # Input size: [batch, 3, 32, 32]
        # Output size: [batch, 3, 32, 32]
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 3, padding=1),            # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.Conv2d(12, 24, 3, padding=1),           # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.Conv2d(24, 48, 3, padding=1),           # [batch, 48, 4, 4]
            nn.ReLU(),
			nn.Conv2d(48, 96, 3, padding=1),           # [batch, 96, 2, 2]
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(96, 48, 3, padding=1),  # [batch, 48, 4, 4]
            nn.ReLU(),
			nn.ConvTranspose2d(48, 24, 3, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 3, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 3, padding=1),   # [batch, 3, 32, 32]
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
def create_conv_model():
    autoencoder = ConvAutoEncoder()
    print_model(autoencoder.encoder, autoencoder.decoder)
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU in order to speed up training.")
    return autoencoder

In [ ]:
conv_encoder = create_conv_model()

# Load data
transform = transforms.Compose(
    [transforms.ToTensor()
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
    ])

custom_set = CustomDataset(1000, transform)
custom_loader = torch.utils.data.DataLoader(custom_set, batch_size=16,
                                          shuffle=True, num_workers=2)

test_set = CustomDataset(1000, transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=16,
                                          shuffle=True, num_workers=2)


In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(conv_encoder.parameters())
epochs = 25

In [ ]:
epoch_counter = 0
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, _) in enumerate(custom_loader, 0):
        inputs = get_torch_vars(inputs)
        # ============ Forward ============
        encoded, outputs = conv_encoder(inputs)
        loss = criterion(outputs, inputs)
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.6f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    epoch_counter += 1
    if epoch_counter == 4:
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(conv_encoder.state_dict(), "./weights/conv_encoder.pkl")
        epoch_counter = -1
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(conv_encoder.state_dict(), "./weights/conv_encoder.pkl")

In [ ]:
class DeepConvAutoEncoder(nn.Module):
    def __init__(self, encoder):
        super(DeepConvAutoEncoder, self).__init__()
        self.encoder = encoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(96, 72, 3, padding=1),  # [batch, 48, 4, 4]
            nn.ReLU(),
            nn.ConvTranspose2d(72, 60, 3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(60, 48, 3, padding=1),
            nn.ReLU(),
			nn.ConvTranspose2d(48, 36, 3, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
            nn.ConvTranspose2d(36, 24, 3, padding=1),
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 3, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 6, 3, padding=1),   # [batch, 3, 32, 32]
            nn.ReLU(),
            nn.ConvTranspose2d(6, 3, 3, padding=1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
def create_deep_model(encoder):
    autoencoder = DeepConvAutoEncoder(encoder)
    print_model(autoencoder.encoder, autoencoder.decoder)
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU in order to speed up training.")
    return autoencoder

In [ ]:
deep_autoencoder1 = create_deep_model(conv_encoder.encoder)

In [ ]:
for index, param in enumerate(deep_autoencoder1.parameters()):
    if index < 8:
        param.requires_grad = False

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, deep_autoencoder1.parameters()))
epochs = 10

In [ ]:
epoch_counter = 0
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, (outputs1, outputs2)) in enumerate(custom_loader, 0):
        inputs = get_torch_vars(inputs)
        outputs1 = get_torch_vars(outputs1)
        outputs2 = get_torch_vars(outputs2)
        # ============ Forward ============
        encoded, outputs = deep_autoencoder1(inputs)
        loss = criterion(outputs, outputs1)
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # ============ Logging ============
        running_loss += loss.data
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    if epoch_counter == 4:
        epoch_counter = -1
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(deep_autoencoder1.state_dict(), "./weights/deep_autoencoder1.pkl")
    epoch_counter += 1
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(deep_autoencoder1.state_dict(), "./weights/deep_autoencoder1.pkl")